In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cityblock
from scipy.optimize import minimize
from sklearn import datasets

In [2]:
from sklearn import svm, datasets
from sklearn.model_selection import cross_val_score
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf = svm.SVC(gamma='scale', random_state=0)
cross_val_score(clf, X, y, scoring='recall_macro',cv=5)  

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

# Data prep

In [3]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [4]:
#np.random.shuffle(X)
split=0.8
X_train,y_train=X[:int(split*len(X)),:],y[:int(split*len(X))]
X_test,y_test=X[int(split*len(X)):,:],y[int(split*len(X)):]
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(120, 4) (120,)
(30, 4) (30,)


# Train 

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [6]:
_calculate_franges(X_train)

NameError: name '_calculate_franges' is not defined

In [7]:
#parameters = {'hidden_layer_sizes':((5,),(10,)),'activation':('relu','logistic')}
#nn = MLPClassifier()
#clf = GridSearchCV(nn, parameters, cv=2,scoring='accuracy')

In [8]:
#clf.fit(X_train,y_train)

In [9]:
#clf.cv_results_

In [10]:
#clf = LogisticRegression()
clf = svm.SVC(gamma='scale',probability=True)
#cross_val_score(clf, X, y, scoring='recall_macro',cv=5)
clf.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
preds=clf.predict_proba(X)

In [12]:
preds

array([[0.97834011, 0.01344511, 0.00821478],
       [0.97275694, 0.01811364, 0.00912942],
       [0.97847865, 0.01251508, 0.00900627],
       [0.97049768, 0.01909167, 0.01041065],
       [0.97918648, 0.0124115 , 0.00840202],
       [0.96186742, 0.02701791, 0.01111468],
       [0.97640315, 0.01382624, 0.00977061],
       [0.97515233, 0.01616953, 0.00867814],
       [0.96828025, 0.01961577, 0.01210398],
       [0.9721749 , 0.01856692, 0.00925818],
       [0.97356002, 0.01741541, 0.00902456],
       [0.97167901, 0.01859543, 0.00972557],
       [0.97404183, 0.01658655, 0.00937162],
       [0.97606303, 0.01146288, 0.01247409],
       [0.97194199, 0.01704967, 0.01100835],
       [0.96048587, 0.02586656, 0.01364756],
       [0.97634571, 0.01450332, 0.00915096],
       [0.97732131, 0.01438238, 0.00829631],
       [0.95528576, 0.03291139, 0.01180285],
       [0.9756705 , 0.01520921, 0.00912029],
       [0.95957175, 0.03009447, 0.01033379],
       [0.97400134, 0.01689075, 0.00910791],
       [0.

In [13]:
confusion_matrix(y,preds)

ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [274]:
recall_score(y,preds,average='macro')

0.9866666666666667

In [278]:
1/len(np.unique(y))

0.3333333333333333

# Explainer class

In [54]:
def _calculate_franges(X_train):
    f_ranges=[]
    for i in range(X_train.shape[1]):
        mi,ma=X_train[:,i].min(),X_train[:,i].max()
        f_ranges.append((mi,ma))
    return f_ranges


def _calculate_radius(f_ranges,epsilon=0.1):
    rs=[]
    for l,h in f_ranges:
        r=epsilon*(h-l)
        rs.append(r)
    return rs


def _generate_samples(X,rs,nb_samples,all_positive=True):
    lower_bounds = X-rs
    upper_bounds = X+rs
    if all_positive:
        lower_bounds[lower_bounds<0]=0
            
    samples_in=[]
    for _ in range(nb_samples):
        S=np.asarray([np.random.uniform(low=lb,high=hb) for lb,hb in zip(lower_bounds,upper_bounds)])
        #if cityblock(S,X) < epsilon:
        #    samples_in.append(S)
        samples_in.append(S)
    return np.asarray(samples_in)


def _calculate_confidence_treshold(X,model,y_train):
    preds=model.predict(X)
    assert isinstance(pred,np.array), 'predictions not in a np.array format. Prediction format: {}'.format(type(preds))
    pred_class=np.argmax(preds)
    p_class = len(y_train[np.where(y_train==pred_class)])/len(y_train)
    return 1-p_class
    
        
class CounterFactual(object):
    
    def __init__(self,model,X_train,y_train,
                 method='rndsample',tollerance=0,epsilon=0.1,epsilon_step=0.1,lam=1,nb_samples=100):
        self.model=model
        self.X_train=X_train
        self.y_train=y_train
        self.method=method
        self.tollerance=tollerance
        self.epsilon=epsilon
        self.epsilon_step=epsilon_step
        self.lam=lam
        self.nb_samples=nb_samples
        
    def generate_explanation(self,X):
        f_ranges=_calculate_franges(self.X_train)
        probas_x = self.model.predict_proba(np.asarray(X).reshape(1, -1))
        pred_class = np.argmax(probas_x,axis=1)[0]
        max_proba_x = probas_x[:,pred_class]
        
        if self.method=='rndsample':
            while self.epsilon<2:
                rs=_calculate_radius(f_ranges=f_ranges,epsilon=self.epsilon)
                samples_in=_generate_samples(X,rs,self.nb_samples)
            
                probas_si = self.model.predict_proba(samples_in)
                pred_classes = np.argmax(probas_si,axis=1)
                unique, counts = np.unique(pred_classes, return_counts=True)
                majority_class = unique[np.argmax(counts)]
                print ('Original predicted class: {}; Majority class in sampled data: {}'.format(pred_class, majority_class))                        
                avg_probas = probas_si.mean(axis=0)
            
                if majority_class!=pred_class:
                #if avg_pred<max_proba_x-self.tollerance:
                    samples_flip = samples_in[np.where(pred_classes!=pred_class)]
                    cf_instance=samples_flip.mean(axis=0)
                    print ('Epsilon', self.epsilon)
                    print ('==========================')
                    print ('Number of samples:',len(samples_in))
                    print ('Original predicted class {} with probability {}:'.format(iris.target_names[pred_class],max_proba_x))
                    print ('Majority class in sampled data points', iris.target_names[majority_class])
                    print ('Original instance:',X)
                    print ('Counfact instance:', cf_instance)
                    print ('Features values difference', cf_instance-X)
                    print ('L1 distance from X',cityblock(cf_instance,X))
                    self.explaning_instance=cf_instance
                #self.avg_pred=avg_pred
                    self.samples_flip=samples_flip
                    self.features_values_diff=cf_instance-X
                    self.l1_distance=cityblock(cf_instance,X)
                    break
                
                self.epsilon+=self.epsilon_step
            return self.explaning_instance
        
        elif self.method=='minloss':
            def countefactual_loss(x,XX=np.random.uniform(size=X.shape),lam=self.lam,yy=2/3):
                return lam*(self.model.predict_proba(x.reshape(1,-1))[:,pred_class]-yy)**2 + cityblock(x,XX)
            def contrains_diff(x,yy=2/3,tollerance=self.tollerance):
                return (self.model.predict_proba(x.reshape(1,-1))[:,pred_class]-yy)**2<=tollerance
            def contrains_dist(x,XX=X,epsilon=self.epsilon):
                return cityblock(x,XX)<epsilon
            
            res=minimize(countefactual_loss,X,method='COBYLA',
                         constraints=[{'type':'ineq','fun':contrains_dist},{'type':'ineq','fun':contrains_diff}])
            self.explaning_instance=res.x
            print ('Max proba X',max_proba_x)
            print ('Predicted class original',iris.target_names[pred_class])
            print ('Original instance',X)
            print ('Counfact instance', self.explaning_instance)
            print ('Features values difference', self.explaning_instance-X)
            print ('L1 distance from X',cityblock(self.explaning_instance,X))
            return self.explaning_instance
            

In [55]:
cf=CounterFactual(clf,X,y,method='rndsample',lam=1,tollerance=0.1,epsilon=0.01,nb_samples=300)

In [56]:
expl=cf.generate_explanation(X[20])

Original predicted class: 0; Majority class in sampled data: 0
Original predicted class: 0; Majority class in sampled data: 0
Original predicted class: 0; Majority class in sampled data: 0
Original predicted class: 0; Majority class in sampled data: 0
Original predicted class: 0; Majority class in sampled data: 0
Original predicted class: 0; Majority class in sampled data: 0
Original predicted class: 0; Majority class in sampled data: 1
Epsilon 0.61
Number of samples: 300
Original predicted class setosa with probability [0.95957175]:
Majority class in sampled data points versicolor
Original instance: [5.4 3.4 1.7 0.2]
Counfact instance: [5.4533561  3.30333917 3.7566682  0.8438514 ]
Features values difference [ 0.0533561  -0.09666083  2.0566682   0.6438514 ]
L1 distance from X 2.8505365369914553


In [48]:
expl

array([5.49427191, 3.2487006 , 3.82074829, 0.70802545])

In [428]:
clf.predict([[5.0,3.5,1.3,1.4]])

array([0])

In [429]:
iris.target_names[1]

'versicolor'